In [3]:
import random

In [8]:
class HMM:
    def __init__(self, states, observations, initial_prob, transition_prob, emission_prob):
        """
        Inicializa un modelo de Markov Oculto (HMM) con los parámetros dados.

        Args:
        - states: Lista de estados posibles del HMM.
        - observations: Lista de observaciones posibles del HMM.
        - initial_prob: Diccionario de probabilidades iniciales para cada estado.
        - transition_prob: Diccionario de probabilidades de transición entre estados.
        - emission_prob: Diccionario de probabilidades de emisión para cada estado y observación.
        """
        self.states = states
        self.observations = observations
        self.initial_prob = initial_prob
        self.transition_prob = transition_prob
        self.emission_prob = emission_prob

    def generate_sequence(self, length):
        """
        Genera una secuencia de observaciones basada en el modelo HMM.

        Args:
        - length: Longitud de la secuencia a generar.

        Returns:
        - Secuencia de observaciones generada.
        """
        sequence = []
        # Selecciona un estado inicial basado en las probabilidades iniciales
        current_state = random.choices(self.states, weights=list(self.initial_prob.values()))[0]
        for _ in range(length):
            # Genera una observación basada en las probabilidades de emisión del estado actual
            sequence.append(random.choices(self.observations, weights=list(self.emission_prob[current_state].values()))[0])
            # Transiciona al siguiente estado basado en las probabilidades de transición
            current_state = random.choices(self.states, weights=list(self.transition_prob[current_state].values()))[0]
        return sequence

    def forward(self, observations):
        """
        Calcula las probabilidades forward del modelo HMM.

        Args:
        - observations: Secuencia de observaciones.

        Returns:
        - Lista de diccionarios de probabilidades forward para cada paso de tiempo.
        """
        forward_probs = [{}]
        # Inicializa las probabilidades forward para el primer paso de tiempo
        for state in self.states:
            forward_probs[0][state] = self.initial_prob[state] * self.emission_prob[state][observations[0]]
        # Calcula las probabilidades forward para los pasos de tiempo restantes
        for t in range(1, len(observations)):
            forward_probs.append({})
            for state in self.states:
                # Utiliza la recursión forward para calcular las probabilidades
                forward_probs[t][state] = sum(forward_probs[t-1][prev_state] * self.transition_prob[prev_state][state] * self.emission_prob[state][observations[t]] for prev_state in self.states)
        return forward_probs

    def backward(self, observations):
        """
        Calcula las probabilidades backward del modelo HMM.

        Args:
        - observations: Secuencia de observaciones.

        Returns:
        - Lista de diccionarios de probabilidades backward para cada paso de tiempo.
        """
        backward_probs = [{} for _ in range(len(observations))]
        # Inicializa las probabilidades backward para el último paso de tiempo
        for state in self.states:
            backward_probs[-1][state] = 1
        # Calcula las probabilidades backward para los pasos de tiempo restantes
        for t in range(len(observations) - 2, -1, -1):
            for state in self.states:
                # Utiliza la recursión backward para calcular las probabilidades
                backward_probs[t][state] = sum(self.transition_prob[state][next_state] * self.emission_prob[next_state][observations[t+1]] * backward_probs[t+1][next_state] for next_state in self.states)
        return backward_probs

    def compute_state_probabilities(self, observations):
        """
        Calcula las probabilidades de estado del modelo HMM.

        Args:
        - observations: Secuencia de observaciones.

        Returns:
        - Lista de diccionarios de probabilidades de estado para cada paso de tiempo.
        """
        forward_probs = self.forward(observations)
        backward_probs = self.backward(observations)
        state_probs = [{} for _ in range(len(observations))]
        # Combina las probabilidades forward y backward para calcular las probabilidades de estado
        for t in range(len(observations)):
            total_prob = sum(forward_probs[t][state] * backward_probs[t][state] for state in self.states)
            for state in self.states:
                state_probs[t][state] = (forward_probs[t][state] * backward_probs[t][state]) / total_prob
        return state_probs

# Uso y datos
states = ['Sunny', 'Rainy']
observations = ['Sunny', 'Rainy']  # Agregar 'Rainy' como observación adicional
initial_prob = {'Sunny': 0.4, 'Rainy': 0.6}
transition_prob = {'Sunny': {'Sunny': 0.8, 'Rainy': 0.2}, 'Rainy': {'Sunny': 0.6, 'Rainy': 0.4}}
emission_prob = {'Sunny': {'Sunny': 0.8, 'Rainy': 0.2}, 'Rainy': {'Sunny': 0.7, 'Rainy': 0.3}}

hmm = HMM(states, observations, initial_prob, transition_prob, emission_prob)

# Generar una secuencia de observaciones
obs_sequence = hmm.generate_sequence(5)
print("Secuencia Generada:", obs_sequence)

# Calcular probabilidades forward
forward_probs = hmm.forward(observations)
print("Probabilidades Forward:")
print(forward_probs)

# Calcular probabilidades backward
backward_probs = hmm.backward(observations)
print("Probabilidades Backward:")
print(backward_probs)

# Calcular probabilidades de estado
state_probs = hmm.compute_state_probabilities(observations)
print("Probabilidades de Estados:")
print(state_probs)

Secuencia Generada: ['Sunny', 'Sunny', 'Sunny', 'Sunny', 'Sunny']
Probabilidades Forward:
[{'Sunny': 0.32000000000000006, 'Rainy': 0.42}, {'Sunny': 0.10160000000000002, 'Rainy': 0.06960000000000001}]
Probabilidades Backward:
[{'Sunny': 0.22000000000000003, 'Rainy': 0.24}, {'Sunny': 1, 'Rainy': 1}]
Probabilidades de Estados:
[{'Sunny': 0.4112149532710281, 'Rainy': 0.5887850467289718}, {'Sunny': 0.5934579439252338, 'Rainy': 0.40654205607476634}]


#### Análisis
Las probabilidades calculadas nos dan información sobre la probabilidad de estar en cada estado (Sunny o Rainy) en cada paso de tiempo, dado que se ha observado una secuencia específica de observaciones.

- La secuencia generada es ['Rainy', 'Rainy', 'Sunny', 'Sunny', 'Rainy'], lo que indica que la mayoria de días fueron lluviosos.

- Probabilidades Forward: Estas probabilidades representan la probabilidad de observar la secuencia hasta un paso de tiempo específico y estar en un estado particular en ese paso de tiempo. Por ejemplo, en el primer paso de tiempo, la probabilidad de estar en el estado "Sunny" es aproximadamente 0.32 y en el estado "Rainy" es aproximadamente 0.42.

- Probabilidades Backward: Estas probabilidades representan la probabilidad de observar la secuencia restante desde un paso de tiempo específico hasta el final de la secuencia, dado que el sistema se encuentra en un estado particular en ese paso de tiempo. En el último paso de tiempo, la probabilidad de estar en cualquier estado es 1, ya que estamos en el último paso y conocemos el estado final.

- Probabilidades de Estados: Estas probabilidades combinan las probabilidades hacia adelante y hacia atrás para calcular la probabilidad de estar en cada estado en cada paso de tiempo dado la secuencia observada. Por ejemplo, en el primer paso de tiempo, la probabilidad de estar en el estado "Sunny" es aproximadamente 0.41 y en el estado "Rainy" es aproximadamente 0.59.

Estas probabilidades proporcionan información sobre la distribución de estados a lo largo del tiempo dada la secuencia observada y son útiles para la inferencia en modelos de Markov ocultos.


En resumen, el análisis de estas probabilidades proporciona una comprensión de cómo el modelo HMM interpreta las observaciones y cómo estima la secuencia de estados ocultos que mejor explica esas observaciones. En este caso, el modelo parece capturar bien la alternancia entre estados soleados y lluviosos en la secuencia observada.